<a href="https://colab.research.google.com/github/samiraabnar/representational_similarity/blob/main/gpt_ptb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install datasets
!pip install sentencepiece

     |████████████████████████████████| 2.8 MB 5.1 MB/s 
     |████████████████████████████████| 636 kB 32.2 MB/s 
     |████████████████████████████████| 3.3 MB 34.5 MB/s 
     |████████████████████████████████| 895 kB 41.9 MB/s 
     |████████████████████████████████| 50 kB 6.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 264 kB 5.2 MB/s 
     |████████████████████████████████| 119 kB 47.8 MB/s 
     |████████████████████████████████| 243 kB 37.9 MB/s 
     |████████████████████████████████| 1.2 MB 5.3 MB/s 


In [ ]:
import google
from pprint import pprint

import numpy as np
import torch
from transformers import AutoModel, AutoTokenizer, BertTokenizer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import list_datasets, list_metrics, load_dataset, load_metric
import tqdm

torch.set_grad_enabled(False)

In [ ]:
# Store the model we want to use
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

MODEL_NAMES = [
"openai-gpt",
"gpt2",
"gpt2-medium",
"gpt2-large"
# "gpt2-xl",
# "EleutherAI/gpt-neo-125M",
# "EleutherAI/gpt-neo-1.3B",
# "EleutherAI/gpt-neo-2.7B",
               ] 

models = {}
tokenizers = {}

with torch.no_grad():
  # We need to create the model and tokenizer
  for model_name in MODEL_NAMES:
    print('loading', model_name)
    models[model_name] = AutoModel.from_pretrained(model_name,
                                      output_hidden_states=True,
                                      output_attentions=True).to(device)
    tokenizers[model_name] = AutoTokenizer.from_pretrained(model_name)

loading openai-gpt


Downloading:   0%|          | 0.00/656 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/479M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/816k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/458k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/764 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.25G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
dataset_name = 'ptb_text_only'
dataset_split = 'validation'
# Downloading and loading a dataset
dataset = load_dataset(dataset_name, split=dataset_split)

Downloading:   0%|          | 0.00/2.69k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.70M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/135k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150k [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset ptb_text_only downloaded and prepared to /root/.cache/huggingface/datasets/ptb_text_only/penn_treebank/1.1.0/8d1b97746fb9765d140e569ec5ddd35e20af4d37761f5e1bf357ea0b081f2c1f. Subsequent calls will reuse this data.


In [ ]:
print(f"👉Dataset len(dataset): {len(dataset)}")
print("\n👉First item 'dataset[0]':")
pprint(dataset[0])

👉Dataset len(dataset): 3370

👉First item 'dataset[0]':
{'sentence': 'consumers may want to move their telephones a little closer to '
             'the tv set'}


In [ ]:
 def get_word_idx(sent: str, word: str):
     return sent.split(" ").index(word)
 
 
 def get_hidden_states(encoded, token_ids_word, model, layers, device=device):
     """Push input IDs through model. Stack and sum `layers` (last four by default).
        Select only those subword token outputs that belong to our word of interest
        and average them.""" 
     # Get all hidden states
     inputs = torch.tensor(encoded['input_ids']).to(device)
     attention_mask =  torch.tensor(encoded['attention_mask']).to(device)
     output = model(**encoded)
     states = output.hidden_states
     layers = layers or list(range(len(output.hidden_states)))
     # Stack all requested layers
     # Only select the tokens that constitute the requested word
     output = torch.stack([states[i].detach().squeeze()[token_ids_word] for i in layers])
     word_tokens_output = output.mean(axis=1).detach()
     return word_tokens_output.cpu().numpy()
 
 
 def get_word_vector(sent, tokenizer, model, layers):
     """Get a word vector by first tokenizing the input sentence, getting all token idxs
        that make up the word of interest, and then `get_hidden_states`."""
     encoded = tokenizer.encode_plus(sent, return_tensors="pt")
     for key in encoded:
       encoded[key] = torch.tensor(encoded[key]).to(device)
     # get all token idxs that belong to the word of interest
     hidden_states = {}
     for word_id in set(encoded.word_ids()):
      if word_id is not None:
        token_ids_word = np.where(np.array(encoded.word_ids()) == word_id)
        hidden_states[''.join(np.asarray(encoded.tokens())[token_ids_word]).replace('#', '')] = get_hidden_states(encoded, token_ids_word, model, layers)
     return hidden_states

In [ ]:
sentences = dataset['sentence']

encoded_sentneces = tokenizers[model_name].batch_encode_plus(sentences)
tokenized_sentences = tokenizers[model_name].batch_decode(encoded_sentneces['input_ids'])

print('sentence:', sentences[0])
print('tokenized sentence:', tokenized_sentences[0])

sentence: consumers may want to move their telephones a little closer to the tv set
tokenized sentence: consumers may want to move their telephones a little closer to the tv set


In [ ]:
#@title heatmap plotting

import altair as alt
import numpy as np
import pandas as pd


def similarity_heatmaps(sim_of_sim, labels_dict, axis_title='', width=300, columns=2, min_step=1):
  
  plot_data = pd.DataFrame()
  for key in sim_of_sim:
    # Compute x^2 + y^2 across a 2D grid
    labels = labels_dict[key] or range(len(sim_of_sim[key]))
    x, y = np.meshgrid(labels, labels)
    z = sim_of_sim[key]
    

    # Convert this grid to columnar data expected by Altair
    row = pd.DataFrame({'x': x.ravel(),
                        'y': y.ravel(),
                        'z': sim_of_sim[key].ravel(),
                        'key': key})
    plot_data = plot_data.append(row, ignore_index=True)

  return alt.Chart(plot_data, width=width, height=width).mark_rect().encode(
      x=alt.X('x:N', sort=labels, title=axis_title, 
              axis=alt.Axis(values=np.asarray(labels)[list(range(0, len(labels), min_step))])),
      y=alt.X('y:N', sort=labels, title=axis_title, 
             axis=alt.Axis(values=np.asarray(labels)[list(range(0, len(labels), min_step))])),
      color=alt.Color('z:Q', title='Similarity'),
  ).facet(
      facet=alt.Facet('key:N', title='', header=alt.Header(labelFontSize=16)),
      columns=columns
  ).resolve_scale(
      color='independent',
      x='independent',
      y='independent',
  ).configure_axis(
    labelFontSize=14,
    titleFontSize=16
).configure_legend(
    labelFontSize=14,
    titleFontSize=14
).configure_title(
    fontSize=18)

In [ ]:
#@title layer similarity plotting

import altair as alt
import numpy as np
import pandas as pd


def layer_similarity(sim_of_sim, labels_dict, axis_title='', width=300, columns=2):
  
  plot_data = pd.DataFrame()
  for key in sim_of_sim:
    # Compute x^2 + y^2 across a 2D grid
    labels = labels_dict[key] or list(range(len(sim_of_sim[key])))
    z = sim_of_sim[key].diagonal(1)
    # Convert this grid to columnar data expected by Altair
    row = pd.DataFrame({'x': np.asarray(labels)[:-1],
                        'y': z,
                        'key': key})
    plot_data = plot_data.append(row, ignore_index=True)

  return alt.Chart(plot_data, width=width, height=width).mark_line(point=True).encode(
      x=alt.X('x:O', sort=labels, title='Layer', scale=alt.Scale(zero=False)),
      y=alt.X('y:Q', title='Sim(L(i), L(i+1))', scale=alt.Scale(zero=False)),
      color=alt.Color('key:N', title='Model', scale=alt.Scale(zero=False)),
  ).facet(
      facet=alt.Facet('key:N', title='', header=alt.Header(labelFontSize=16)),
      columns=columns
  ).resolve_scale(
      x='independent',
      y='independent',
  ).configure_axis(
    labelFontSize=14,
    titleFontSize=16
).configure_legend(
    labelFontSize=14,
    titleFontSize=14
).configure_title(
    fontSize=18)

In [ ]:
#@title rsa

import numpy as np
import pandas as pd
from scipy import stats 
from scipy import corrcoef
from scipy.spatial.distance import cosine, euclidean, pdist, squareform, is_valid_dm, cdist
from sklearn.metrics import pairwise_distances
from scipy.stats import spearmanr
from scipy.spatial import distance_matrix

#Visualization packages
import matplotlib.pylab as plt

def pearson_corr(output, target):
  x = output
  y = target

  vx = x - torch.mean(x)
  vy = y - torch.mean(y)

  return torch.sum(vx * vy) / (torch.sqrt(torch.sum(vx ** 2)) * torch.sqrt(torch.sum(vy ** 2)))

def pearson_rsa_for_lists_of_reps(reps):
  """Compute representational similarity between two sets of representations.

  Args:
    reps: list of float arrays; List of arrrays with shape [num_examples, feature_size]
      (feature size can be different for each item in the list).
  """
  norm_reps = []
  for rep in reps:
  # Normalize input representations
    norm_reps.append(rep / torch.linalg.norm(rep, axis=-1)[...,None])

  # Compute and normalize similarity matrices:
  similarity_matrices = []
  for rep in norm_reps:
    similarities = rep.matmul(rep.T) # instead of dot product you can use any similarity measure
    similarities = similarities / (torch.linalg.norm(rep, axis=-1) * torch.linalg.norm(rep, axis=-1))
    similarity_matrices.append(similarities)

  # Shape of similarity_matrices: [num_of_rep_spaces, num_examples, num_examples]
  similarity_matrices = torch.stack(similarity_matrices).cpu().numpy() 
  sim_of_sim_mat = np.zeros((similarity_matrices.shape[0], 
                              similarity_matrices.shape[0]))
  for i in range(sim_of_sim_mat.shape[0]):
    for j in range(sim_of_sim_mat.shape[1]):
      correlation_of_similarities = [stats.pearsonr(a,b)[0] for a, b in zip (similarity_matrices[i], similarity_matrices[j])]
      similarity_score = np.mean(correlation_of_similarities)
      sim_of_sim_mat[i][j] = similarity_score
  return sim_of_sim_mat
  
def dot_product_rsa_for_lists_of_reps(reps):
  """Compute representational similarity between two sets of representations.

  Args:
    reps: list of float arrays; List of arrrays with shape [num_examples, feature_size]
      (feature size can be different for each item in the list).
  """
  norm_reps = []
  for rep in reps:
  # Normalize input representations
    norm_reps.append(rep / torch.linalg.norm(rep, axis=-1)[...,None])

  # Compute and normalize similarity matrices:
  similarity_matrices = []
  for rep in norm_reps:
    similarities = rep.matmul(rep.T) # instead of dot product you can use any similarity measure
    similarities = similarities / (torch.linalg.norm(rep, axis=-1) * torch.linalg.norm(rep, axis=-1))
    similarity_matrices.append(similarities)

  # Shape of similarity_matrices: [num_of_rep_spaces, num_examples, num_examples]
  similarity_matrices = torch.stack(similarity_matrices) 
  sim_of_sim_mat = torch.zeros((similarity_matrices.shape[0], 
                              similarity_matrices.shape[0]))
  for i in range(sim_of_sim_mat.shape[0]):
    for j in range(sim_of_sim_mat.shape[1]):
      # Here you can do pearson-r instead of this (or again any measure of similarity).
      similarity_of_similarity = torch.sum(
          similarity_matrices[i] * similarity_matrices[j], axis=-1) / (
              torch.linalg.norm(similarity_matrices[j], axis=-1) * torch.linalg.norm(similarity_matrices[i], axis=-1))
      similarity_score = torch.mean(similarity_of_similarity)
      sim_of_sim_mat[i][j] = similarity_score
  return sim_of_sim_mat


In [ ]:
import math

def centering(K):
        n = K.shape[0]
        unit = torch.ones([n, n], device=device)
        I = torch.eye(n, device=device)
        H = I - unit / n
        return torch.matmul(torch.matmul(H, K), H)  

def rbf(X, sigma=None):
    GX = torch.matmul(X, X.T)
    KX = torch.diag(GX) - GX + (torch.diag(GX) - GX).T
    if sigma is None:
        mdist = torch.median(KX[KX != 0])
        sigma = math.sqrt(mdist)
    KX *= - 0.5 / (sigma * sigma)
    KX = torch.exp(KX)
    return KX

def kernel_HSIC(X, Y, sigma):
    return torch.sum(centering(rbf(X, sigma)) * centering(rbf(Y, sigma)))

def linear_HSIC(X, Y):
    L_X = torch.matmul(X, X.T)
    L_Y = torch.matmul(Y, Y.T)
    return torch.sum(centering(L_X) * centering(L_Y))

def linear_CKA(X, Y):
    hsic = linear_HSIC(X, Y)
    var1 = torch.sqrt(linear_HSIC(X, X))
    var2 = torch.sqrt(linear_HSIC(Y, Y))

    return hsic / (var1 * var2)

def kernel_CKA(X, Y, sigma=None):
    hsic = kernel_HSIC(X, Y, sigma)
    var1 = torch.sqrt(kernel_HSIC(X, X, sigma))
    var2 = torch.sqrt(kernel_HSIC(Y, Y, sigma))
    return hsic / (var1 * var2)


def linear_cka_rsa_for_lists_of_reps(reps):
  sim_of_sim_mat = torch.zeros((reps.shape[0], 
                              reps.shape[0]))
  for i in range(sim_of_sim_mat.shape[0]):
    for j in range(sim_of_sim_mat.shape[1]):

      similarity_score = linear_CKA(reps[i], reps[j])
      sim_of_sim_mat[i][j] = similarity_score
  return sim_of_sim_mat


def kernel_cka_rsa_for_lists_of_reps(reps):
  sim_of_sim_mat = torch.zeros((reps.shape[0], 
                              reps.shape[0]))
  for i in range(sim_of_sim_mat.shape[0]):
    for j in range(sim_of_sim_mat.shape[1]):
      similarity_score = kernel_CKA(reps[i], reps[j])
      sim_of_sim_mat[i][j] = similarity_score
  return sim_of_sim_mat

In [ ]:
num_of_sentences = 100

word_embeddings = {}
sentence_embeddings = {}

for model_name in MODEL_NAMES:
  word_embeddings[model_name] = {}
  word_embeddings[model_name]['words'] = []
  word_embeddings[model_name]['embeddings'] = []

  sentence_embeddings[model_name] = {}
  sentence_embeddings[model_name]['sentences'] = []
  sentence_embeddings[model_name]['embeddings'] =  []

for model_name in MODEL_NAMES:
  print(model_name)
  for i in range(num_of_sentences):
    embedding_dict = get_word_vector(sentences[i], tokenizers[model_name], models[model_name], layers=None)
    words, embeddings = zip(*embedding_dict.items())
    word_embeddings[model_name]['words'].extend(words)
    word_embeddings[model_name]['embeddings'].extend(embeddings)

    sentence_embeddings[model_name]['sentences'].append(sentences[i])
    sentence_embeddings[model_name]['embeddings'].append(embeddings)

openai-gpt


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()


gpt2
gpt2-medium
gpt2-large


In [ ]:
words_linear_cka_sim_of_sim = {}
labels_dict = {}
with torch.no_grad():
  for model_name in MODEL_NAMES:
    em_array = np.asarray(
            word_embeddings[model_name]['embeddings']).transpose((1,0,2))
    em_array = torch.tensor(em_array, device=device)
    print(em_array.shape)
    words_linear_cka_sim_of_sim[model_name] = linear_cka_rsa_for_lists_of_reps(em_array)
    labels_dict[model_name] = [f'L{i}' for i in range(em_array.shape[0])]
    del em_array
    torch.cuda.empty_cache()

torch.Size([13, 2057, 768])
torch.Size([13, 2187, 768])
torch.Size([25, 2187, 1024])
torch.Size([37, 2187, 1280])


In [ ]:
similarity_heatmaps(words_linear_cka_sim_of_sim, labels_dict, axis_title='Layer', columns=2, min_step=4)

alt.FacetChart(...)

In [ ]:
layer_similarity(words_linear_cka_sim_of_sim, labels_dict, width=200, columns=2)

alt.FacetChart(...)

In [ ]:
from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection

from sklearn import manifold
from sklearn.metrics import euclidean_distances
from sklearn.decomposition import PCA

source = pd.DataFrame()
for model_name in words_linear_cka_sim_of_sim.keys():
  Lz = [np.int(l.split('L')[-1]) for l in labels_dict[model_name]]
  seed = np.random.RandomState(seed=10)
  mds = manifold.MDS(n_components=2, 
                      #max_iter=3000, 
                      #eps=1e-9, random_state=seed,
                      dissimilarity="precomputed", n_jobs=1)
  pos = mds.fit(1.0 - words_linear_cka_sim_of_sim[model_name]).embedding_
  # pos *= np.sqrt(((1.0 - sentence_sim_of_sim[model_name]) ** 2).sum()) / np.sqrt((pos ** 2).sum())

  source = source.append(pd.DataFrame({
      'x': pos[:, 0],
      'y': pos[:, 1],
      'label': labels_dict[model_name],
      'model name': model_name,
      'layer': Lz,
  }))


points = alt.Chart(source, width=200, height=200).mark_point().encode(
    x=alt.X('x:Q', title=''),
    y=alt.Y('y:Q', title=''),
    # color=alt.Color('model name:N'),
    # # color=alt.Color('layer:O'),
    color=alt.Color('layer:Q', legend=None, 
                    scale=alt.Scale(scheme='blues', domain=(-30,30))),
    size=alt.Size('layer:Q', scale=alt.Scale(range=(10,200)), title='Layer'),


).facet(
    facet=alt.Facet('model name', title='Model Name', header=alt.Header(labelFontSize=16)),
    columns=2
).resolve_scale(
    x='independent',
    y='independent',
    color='independent'
)

# text = points.mark_text(
#     align='left',
#     baseline='middle',
#     dx=7
# ).encode(
#     text='label'
# )

points.configure_axis(
    labelFontSize=14,
    titleFontSize=18
).configure_legend(
    labelFontSize=14,
    titleFontSize=14
).configure_title(
    fontSize=18
)

alt.FacetChart(...)

In [ ]:
print('model_name mean_sim var_sim')
for model_name in MODEL_NAMES:
    mean_sim = torch.mean(words_linear_cka_sim_of_sim[model_name])
    var_sim = torch.var(words_linear_cka_sim_of_sim[model_name])
    print(model_name, mean_sim, var_sim)

model_name mean_sim var_sim
openai-gpt tensor(0.7196) tensor(0.0314)
gpt2 tensor(0.8709) tensor(0.0613)
gpt2-medium tensor(0.9019) tensor(0.0684)
gpt2-large tensor(0.8600) tensor(0.0544)


In [ ]:
print('model_name laye mean_sim var_sim')
for model_name in MODEL_NAMES:
    for l in range(words_linear_cka_sim_of_sim[model_name].shape[0]):
        mean_sim = torch.mean(words_linear_cka_sim_of_sim[model_name][l])
        var_sim = torch.var(words_linear_cka_sim_of_sim[model_name][l])
        print(model_name, l, mean_sim, var_sim)

model_name laye mean_sim var_sim
openai-gpt 0 tensor(0.5545) tensor(0.0252)
openai-gpt 1 tensor(0.6681) tensor(0.0231)
openai-gpt 2 tensor(0.7564) tensor(0.0259)
openai-gpt 3 tensor(0.7345) tensor(0.0336)
openai-gpt 4 tensor(0.7520) tensor(0.0339)
openai-gpt 5 tensor(0.7943) tensor(0.0251)
openai-gpt 6 tensor(0.7935) tensor(0.0226)
openai-gpt 7 tensor(0.7995) tensor(0.0217)
openai-gpt 8 tensor(0.7927) tensor(0.0202)
openai-gpt 9 tensor(0.7574) tensor(0.0247)
openai-gpt 10 tensor(0.7105) tensor(0.0307)
openai-gpt 11 tensor(0.6850) tensor(0.0316)
openai-gpt 12 tensor(0.5559) tensor(0.0297)
gpt2 0 tensor(0.7884) tensor(0.0254)
gpt2 1 tensor(0.9173) tensor(0.0362)
gpt2 2 tensor(0.9279) tensor(0.0406)
gpt2 3 tensor(0.9275) tensor(0.0407)
gpt2 4 tensor(0.9276) tensor(0.0406)
gpt2 5 tensor(0.9277) tensor(0.0406)
gpt2 6 tensor(0.9278) tensor(0.0405)
gpt2 7 tensor(0.9278) tensor(0.0405)
gpt2 8 tensor(0.9278) tensor(0.0406)
gpt2 9 tensor(0.9278) tensor(0.0406)
gpt2 10 tensor(0.9278) tensor(0.040

In [ ]:
mean_sentence_embeddings = {}

for model_name in MODEL_NAMES:
    mean_sentence_embeddings[model_name] = []
    for i in range(len(sentence_embeddings[model_name]['embeddings'])):
      mean_sentence_embeddings[model_name].append(np.mean(np.asarray(sentence_embeddings[model_name]['embeddings'][i]), axis=0))

In [ ]:
mean_sentence_kernel_cka_sim_of_sim = {}
labels_dict = {}
for model_name in MODEL_NAMES:
  em_array = np.asarray(
          mean_sentence_embeddings[model_name]).transpose((1,0,2))
  em_array = torch.tensor(em_array, device=device)
  mean_sentence_kernel_cka_sim_of_sim[model_name] = kernel_cka_rsa_for_lists_of_reps(em_array)
  labels_dict[model_name] = [f'L{i}' for i in range(em_array.shape[0])]
  del em_array
  torch.cuda.empty_cache()


In [ ]:
model_embeddings = []
keys = []
for model_name in MODEL_NAMES:
#   if model_name != 'openai-gpt':
  for layer in range(len(mean_sentence_embeddings[model_name][0])):
    em_array = np.asarray(
            mean_sentence_embeddings[model_name]).transpose((1,0,2))
    em_array = torch.tensor(em_array, device=device)[layer]
    model_embeddings.append(em_array)
    print(f'{model_name}_L{layer}')
    keys.append(f'{model_name}_L{layer}')


model_similarity = linear_cka_rsa_for_lists_of_reps(np.asarray(model_embeddings))

openai-gpt_L0
openai-gpt_L1
openai-gpt_L2
openai-gpt_L3
openai-gpt_L4
openai-gpt_L5
openai-gpt_L6
openai-gpt_L7
openai-gpt_L8
openai-gpt_L9
openai-gpt_L10
openai-gpt_L11
openai-gpt_L12
gpt2_L0
gpt2_L1
gpt2_L2
gpt2_L3
gpt2_L4
gpt2_L5
gpt2_L6
gpt2_L7
gpt2_L8
gpt2_L9
gpt2_L10
gpt2_L11
gpt2_L12
gpt2-medium_L0
gpt2-medium_L1
gpt2-medium_L2
gpt2-medium_L3
gpt2-medium_L4
gpt2-medium_L5
gpt2-medium_L6
gpt2-medium_L7
gpt2-medium_L8
gpt2-medium_L9
gpt2-medium_L10
gpt2-medium_L11
gpt2-medium_L12
gpt2-medium_L13
gpt2-medium_L14
gpt2-medium_L15
gpt2-medium_L16
gpt2-medium_L17
gpt2-medium_L18
gpt2-medium_L19
gpt2-medium_L20
gpt2-medium_L21
gpt2-medium_L22
gpt2-medium_L23
gpt2-medium_L24
gpt2-large_L0
gpt2-large_L1
gpt2-large_L2
gpt2-large_L3
gpt2-large_L4
gpt2-large_L5
gpt2-large_L6
gpt2-large_L7
gpt2-large_L8
gpt2-large_L9
gpt2-large_L10
gpt2-large_L11
gpt2-large_L12
gpt2-large_L13
gpt2-large_L14
gpt2-large_L15
gpt2-large_L16
gpt2-large_L17
gpt2-large_L18
gpt2-large_L19
gpt2-large_L20
gpt2-large_L2

In [ ]:
alt.data_transformers.disable_max_rows()
similarity_heatmaps({'Models Similarity': model_similarity}, 
                    {'Models Similarity': keys}, width=600, min_step=6
                   ).configure_axis(
    labelFontSize=14,
    titleFontSize=16
).configure_legend(
    labelFontSize=14,
    titleFontSize=14
).configure_title(
    fontSize=18
)


alt.FacetChart(...)

In [ ]:
from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection

from sklearn import manifold
from sklearn.metrics import euclidean_distances
from sklearn.decomposition import PCA

In [ ]:
seed = np.random.RandomState(seed=10)
mds = manifold.MDS(n_components=2, max_iter=3000, eps=1e-9, random_state=seed,
                   dissimilarity="precomputed", n_jobs=1)
pos = mds.fit(1.0 - model_similarity).embedding_


source = pd.DataFrame({
    'x': pos[:, 0],
    'y': pos[:, 1],
    'label': keys,
    'model name': [key.split('_')[0] for key in keys],
    'layer': [int(key.split('_L')[1]) for key in keys],
})

points = alt.Chart(source).mark_point().encode(
    x='x:Q',
    y='y:Q',
    color=alt.Color('model name:N', title='Model Name', scale=alt.Scale(scheme='dark2')),
    # color=alt.Color('layer:O'),
    size=alt.Size('layer:Q', sort=[key.split('_L')[1] for key in keys], title='Layer',
                  scale=alt.Scale(type='linear'))

)

line = alt.Chart(source).mark_line(strokeDash=(2,2)).encode(
    x='x:Q',
    y='y:Q',
    color=alt.Color('model name:N', title='Model Name'),
    # color=alt.Color('layer:O'),
    # size=alt.Size('layer:N', sort=[key.split('_L')[1] for key in keys], title='Layer')

)

text = points.mark_text(
    align='left',
    baseline='middle',
    dx=7
).encode(
    text='label'
)

points.configure_axis(
    labelFontSize=14,
    titleFontSize=16
).configure_legend(
    labelFontSize=14,
    titleFontSize=14
).configure_title(
    fontSize=18
)


alt.Chart(...)